In [246]:
# import required packages
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# import required csv files
train = pd.read_csv('/Users/shuai/Desktop/AIPI 520/Kaggle Competitiom/for_students/train.csv')
subscriptions = pd.read_csv('/Users/shuai/Desktop/AIPI 520/Kaggle Competitiom/for_students/subscriptions.csv')
account = pd.read_csv('/Users/shuai/Desktop/AIPI 520/Kaggle Competitiom/for_students/account.csv', encoding='latin1')
test = pd.read_csv('/Users/shuai/Desktop/AIPI 520/Kaggle Competitiom/for_students/test.csv')

# change the name of target columns in the test file for the use of merge function
test.rename(columns={'ID': 'account.id'}, inplace=True)



In [247]:
 
# Create a function to convert True/False values into 1/0
def replace_boolean_with_binary(df):
    # Create a list of columns that contain True/False values
    boolean_columns = [col for col in df.columns if df[col].dtype == bool]
    # Define the replacement dictionary
    replacement_dict = {True: 1, False: 0}
    # Loop through the boolean columns and apply the replacement
    for col in boolean_columns:
        df[col] = df[col].replace(replacement_dict)
account.head()

,account.id,shipping.zip.code,billing.zip.code,shipping.city,billing.city,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,first.donated
0,001i000000LhSrQ,94102,94102,San Francisco,San Francisco,NaN,0.0,0.0,0,NaN
1,001i000000LhyPD,NaN,94303,NaN,Palo Alto,NaN,0.0,500.0,5,1987-07-04 00:00:00
2,001i000000LhyPE,NaN,94597,NaN,Walnut Creek,NaN,0.0,783.0,7,2008-06-02 00:00:00
3,001i000000LhyPF,NaN,94610,NaN,Oakland,NaN,1500.0,28435.0,28,1986-07-04 00:00:00
4,001i000000LhyPG,NaN,94024,NaN,Los Altos,NaN,300.0,4969.0,16,1997-04-07 00:00:00


In [248]:
# drop the irrelevant columns in account.csv
acc_dropped = account[['account.id', 'no.donations.lifetime']]

# combine the dropped dateframe with train set and test set
train_acc = train.merge(acc_dropped, on = 'account.id', how ='left')
test_acc = test.merge(acc_dropped, on = 'account.id', how ='left')



In [249]:
# avoid duplicates by calculating the sums of numerical values and most frequent values of categorial values of subscriptions.csv
aggregated_subscriptions = subscriptions.groupby('account.id').agg({
    'season': lambda x: x.mode()[0] if not x.mode().empty else None,
    'package': lambda x: x.mode()[0] if not x.mode().empty else None,
    'no.seats': 'sum',
    'section': lambda x: x.mode()[0] if not x.mode().empty else None,
    'location': lambda x: x.mode()[0] if not x.mode().empty else None,
    'price.level': 'sum',
    'subscription_tier': lambda x: x.mode()[0] if not x.mode().empty else None,
    'multiple.subs': lambda x: x.mode()[0] if not x.moda e().empty else None,
}).reset_index()

# get dummies for useful features and drop the irrelevant features
dummy_subscriptions = pd.get_dummies(aggregated_subscriptions, columns = ['location', 'section', 'season'])
clean_subscriptions = dummy_subscriptions.drop(['multiple.subs', 'package', 'price.level'], axis = 1)

# convert True/False values into 1/0
replace_boolean_with_binary(clean_subscriptions)

# combine the result with train set and test set and drop nah values
train_acc_subscriptions = train_acc.merge(clean_subscriptions, on = 'account.id', how = 'left')
train_acc_subscriptions = train_acc_subscriptions.fillna(0)

test_acc_subscriptions = test_acc.merge(clean_subscriptions, on = 'account.id', how = 'left')
test_acc_subscriptions = test_acc_subscriptions.fillna(0)



SyntaxError: expected 'else' after 'if' expression (2762618616.py, line 10)

In [ ]:
# define the indices required for the model
x_train = train_acc_subscriptions.drop(columns = ['account.id', 'label'])
y_train = train_acc_subscriptions['label']
x_test = test_acc_subscriptions.drop(columns = ['account.id'])

# define and run the model, get the second element of resulting list
model = LogisticRegression()
model.fit(x_train, y_train)
y_test = model.predict_proba(x_test)
result = [item[1] for item in y_test]


/Users/shuai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# create a new column to store the result and rename account.id to ID
test['predicted'] = result
test.rename(columns={'account.id': 'ID'}, inplace=True)

# save it to csv file 
test.to_csv('test_with_predictions.csv', index=False)

